In [5]:
import csv
import json
import sys
import pandas as pd
import requests
from urllib.parse import urlparse, unquote
import time
from pathlib import Path

In [7]:
metadata = pd.read_csv("./metadata.csv", encoding="latin-1")
train = pd.read_csv("./train_QA.csv", encoding="utf-8")
test = pd.read_csv("./test_Q.csv", encoding="utf-8")
#metadata.head()
#train.head()
#test.head()
FOLDER = "./data"


In [4]:
URL_LIST = metadata['url']
#HTTP headers
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

for index, url in enumerate(URL_LIST):
	url = url.strip()
	try:
		filename = f"file_{index + 1}.pdf"
		if not Path(FOLDER).exists():
			print(f"{FOLDER} not found")
			break
		save_path = Path(FOLDER) / filename

		response = requests.get(url, headers=headers, stream=True, timeout=30)
		# throw an exception
		response.raise_for_status()

		with open(save_path, 'wb') as f:
			for chunk in response.iter_content(chunk_size=8192):
				f.write(chunk)

		time.sleep(0.5)

	except Exception as e:
		print(f"[fail] ({index+1}/{len(URL_LIST)}) URL: {url}")
		print(f"       reason: {e}")



In [8]:
from numpy.linalg import norm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
import torch
import shutil

from langchain_community.llms import Ollama
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import JinaEmbeddings
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain_classic.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, UnstructuredPDFLoader
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer
from langchain_ollama import OllamaLLM
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cengy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\cengy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from huggingface_hub import login

hf_token = "..."
login(token=hf_token, add_to_git_credential=True)

In [8]:
!hf auth whoami

user:  a1n2d2y8


In [ ]:
#!ollama pull llama3.2:3b

In [11]:
MODEL = "llama3.2:3b" 
# Initialize an instance of the Ollama model
llm = OllamaLLM(model=MODEL)
# Invoke the model to generate responses
response = llm.invoke("What is the capital of Taiwan?")
print(response)

The capital of Taiwan is Taipei.


In [13]:
DB_FOLDER = "./chroma_db_store"
device = "cuda"

if Path(DB_FOLDER).exists():
	shutil.rmtree(Path(DB_FOLDER))
Path(DB_FOLDER).mkdir(parents=True, exist_ok=True)

loader = DirectoryLoader(
	FOLDER, 
	glob="*.pdf", 
	loader_cls=PyPDFLoader,
)
print("loading PDF...")
documents = loader.load()
if not documents:
	print("no file")
	
# Splitting
text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True
)
print("splitting...")
splits = text_splitter.split_documents(documents)

model_kwargs = {
	'device': device, 
	'trust_remote_code': True
}
encode_kwargs = {'normalize_embeddings': False}

loading PDF...
splitting...


In [14]:
# Embedding Model Setup
EMBED_MODEL = "intfloat/multilingual-e5-large"
embeddings_model = HuggingFaceEmbeddings(
	model_name=EMBED_MODEL,
	model_kwargs=model_kwargs,
	encode_kwargs=encode_kwargs
)

In [15]:
# Creating Vector Store
try:
    vector_store.delete_collection()
    print("old Collection clear")
except:
    pass
vector_store = Chroma.from_documents(
	documents=splits,
	embedding=embeddings_model,
	persist_directory=DB_FOLDER
)
retriever = vector_store.as_retriever(
	search_type="similarity",
	search_kwargs={"k": 3}
)

In [24]:
print("test...")
query = "What is the name of the benchmark suite presented in a recent paper for measuring inference energy consumption?" 
results = retriever.invoke(query)

for i, doc in enumerate(results):
	print(f"\nSource{i+1}: {doc.metadata.get('source')}")
	print(f"Content: {doc.page_content[:1000]}...") 

test...

Source1: data\file_3.pdf
Content: On July 2023, we launched the ML.ENERGY Leaderboard and Benchmark, the first inference energy
leaderboard for modern generative AI models.6 Our goal was to measure and understand the energy
consumption of generative AI models, and we provided a web-based leaderboard to allow everyone
to browse the results. The leaderboard started with only LLM chat with tens of different LLMs, but
gradually expanded to include more tasks, models, and datasets. Our benchmarking suite to supply
data to the leaderboard is what we dub the ML.ENERGY Benchmark. This paper shares our design
philosophy and principles we have acquired over time by gradually maintaining and upgrading the
ML.ENERGY Benchmark and the Leaderboard, and highlights notable results we have obtained
from the early 2025 iteration of the benchmark. Importantly, we plan to continuously update the
benchmark and the leaderboard as long as resources allow, and what is presented in this paper is only.

In [ ]:
import os
import pandas as pd
import time
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
CSV_PATH = "./train_QA.csv"      
OUTPUT_CSV = "./answer.csv" 
def clean_json_string(json_str):
    json_str = json_str.strip()
    if json_str.startswith("```json"):
        json_str = json_str[7:]
    if json_str.startswith("```"):
        json_str = json_str[3:]
    if json_str.endswith("```"):
        json_str = json_str[:-3]
    return json_str.strip()
template_str = """
    You are a data extraction expert. Please answer the questions based on the provided "Reference Documents.
    
    Please be sure to output in **standard JSON format**, without including any other text or interpretations.
    The JSON must contain the following fields:
    - "answer": A clear natural-language response (e.g., 1438 lbs, Water consumption, TRUE).
        * If no answer is possible, use exactly: "Unable to answer with confidence based on the provided documents."
    - "answer_value": The normalized numeric or categorical value (e.g., 1438, Water consumption, 1).
        * If no answer is possible, use "is_blank".
        * Ranges should be encoded as "[low, high]".
        * Do not include symbols like <, >, ~ here. Those can be left in the "answer" field.
    - "answer_unit": Unit of measurement (e.g., lbs, kWh, gCO2, projects, is_blank).
    - "supporting_materials": Verbatim justification from the cited document (quote, table reference, figure reference, etc.).
    - "explanation": Short reasoning describing why the cited material supports the answer.
    
    If you cannot find the answer in the reference file, please enter "Unable to answer with confidence based on the provided documents." in the answer field.

    === reference file ===
    {context}
    === question ===
    {question}
    
    JSON output:
    """

PROMPT = PromptTemplate(
    template=template_str, 
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT} 
)

def run_testing():

    if not os.path.exists(DB_FOLDER):
        print("no database")
        return
    df = pd.read_csv(CSV_PATH)

    results = []

    for index, row in df.iterrows():
        question = row['question']
        qid = row.get('id', index)
        
        print(f"\n[ID: {qid}] generating...")

        row_result = {
            "id": qid,
            "question": question,
            "answer": "Error",
            "answer_value": "",
            "answer_unit": "",
            "supporting_materials": "",
            "explanation": "",
            "ref_url": "",   
            "ref_id": ""   
        }

        try:
            response = qa_chain.invoke({"query": question})
            raw_output = response['result']
            source_docs = response['source_documents']

            try:
                cleaned_json = clean_json_string(raw_output)
                parsed_data = json.loads(cleaned_json)
                
                row_result["answer"] = parsed_data.get("answer", "")
                row_result["answer_value"] = parsed_data.get("answer_value", "")
                row_result["answer_unit"] = parsed_data.get("answer_unit", "")
                row_result["supporting_materials"] = parsed_data.get("supporting_materials", "")
                row_result["explanation"] = parsed_data.get("explanation", "")

            except json.JSONDecodeError:
                print("   -> JSON decode error")
                row_result["answer"] = raw_output 

            if source_docs:
                source_path = source_docs[0].metadata.get('source', '')
                filename = os.path.basename(source_path) 
                

                row_result["ref_url"] = filename
                
                row_result["ref_id"] = filename.replace(".pdf", "")

        except Exception as e:
            print(f"   -> RAG fail: {e}")
            row_result["explanation"] = f"System Error: {e}"

        results.append(row_result)
        time.sleep(0.5)

    out_df = pd.DataFrame(results)
    
    cols_order = ["id", "question", "answer", "answer_value", "answer_unit", "ref_id", "ref_url", "supporting_materials", "explanation"]

    final_cols = [c for c in cols_order if c in out_df.columns]
    out_df = out_df[final_cols]

    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')

if __name__ == "__main__":
    run_testing()


[ID: q003] generating...

[ID: q009] generating...

[ID: q054] generating...

[ID: q062] generating...

[ID: q075] generating...

[ID: q078] generating...
   -> JSON decode error

[ID: q091] generating...

[ID: q102] generating...

[ID: q105] generating...

[ID: q106] generating...

[ID: q124] generating...

[ID: q135] generating...

[ID: q139] generating...

[ID: q146] generating...

[ID: q153] generating...

[ID: q158] generating...

[ID: q164] generating...

[ID: q166] generating...

[ID: q170] generating...

[ID: q200] generating...

[ID: q202] generating...
   -> JSON decode error

[ID: q203] generating...

[ID: q207] generating...

[ID: q211] generating...

[ID: q215] generating...

[ID: q221] generating...

[ID: q230] generating...

[ID: q231] generating...

[ID: q246] generating...

[ID: q253] generating...

[ID: q262] generating...

[ID: q263] generating...

[ID: q272] generating...

[ID: q278] generating...

[ID: q280] generating...

[ID: q282] generating...

[ID: q296] gene

In [22]:
import pandas as pd
def calculate_accuracy(evaluation_csv_path, train_csv_path):	
    evaluation_report = pd.read_csv(evaluation_csv_path)
    train = pd.read_csv(train_csv_path)
    
    if len(evaluation_report) != len(train):
        raise ValueError("length not match")
    
    results = []
    for eval_ans, true_ans in zip(evaluation_report['answer'], train['answer']):
        if str(true_ans).strip().lower() in str(eval_ans).strip().lower():
            results.append(True)
        else:
            results.append(False)
    
    accuracy = sum(results) / len(results)
    return accuracy, results


accuracy, results = calculate_accuracy("answer.csv", "train_QA.csv")
print("accuracy:", accuracy)
print("outcome:", results)

accuracy: 0.36585365853658536
outcome: [False, True, False, True, True, False, False, True, False, False, False, True, False, True, True, False, True, False, False, False, False, True, True, False, False, True, False, False, False, False, False, True, False, True, False, True, False, False, False, True, False]
